# Tensorflow Implementation of VGG Face Detector

Objective: Convert the VGG face descriptor model http://www.robots.ox.ac.uk/~vgg/software/vgg_face/ to Tensorflow format.
<br>
Output: Final Tensorflow classifier model trained on the gender dataset

In [1]:
import scipy.io as sio
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Reshape, ZeroPadding2D, Convolution2D, MaxPooling2D, Dense, Dropout, Softmax, Flatten, Activation, BatchNormalization

Using TensorFlow backend.


## Read .mat file for Model Description

In [2]:
def load_mat():
    mat = sio.loadmat('data/vgg_face.mat', struct_as_record=False)
    return mat

mat = load_mat()
net = mat['net'][0][0]
mat_model = net.layers
mat_model_layers = mat_model[0]
num_mat_layers = mat_model_layers.shape[0]
for layer in mat_model_layers:
    print(layer[0][0].name)

['conv1_1']
['relu1_1']
['conv1_2']
['relu1_2']
['pool1']
['conv2_1']
['relu2_1']
['conv2_2']
['relu2_2']
['pool2']
['conv3_1']
['relu3_1']
['conv3_2']
['relu3_2']
['conv3_3']
['relu3_3']
['pool3']
['conv4_1']
['relu4_1']
['conv4_2']
['relu4_2']
['conv4_3']
['relu4_3']
['pool4']
['conv5_1']
['relu5_1']
['conv5_2']
['relu5_2']
['conv5_3']
['relu5_3']
['pool5']
['fc6']
['relu6']
['dropout6']
['fc7']
['relu7']
['dropout7']
['fc8']
['softmax']


## VGG Architecture Implementation
We can read the model described in the .mat file and build our tensorflow/keras model using clues. This way we can avoid reproducing it by hand. Additionally, this saves us the trouble of later applying the model weights from the .mat file to our tensorflow/keras model.

In [3]:
def vgg_tf():
    model = Sequential()

    for i in range(num_mat_layers):
        mat_model_layer = mat_model_layers[i][0][0].name[0]
        if mat_model_layer.find("conv") == 0 or mat_model_layer.find("fc") == 0:
            weights = mat_model_layers[i][0,0].weights
            weights_shape = weights[0][0].shape
            filter_x = weights_shape[0]; filter_y = weights_shape[1]
            number_of_filters = weights_shape[3]

            if mat_model_layer.find("conv") == 0:
                print("ZeroPadding2D((1,1))")
                if i == 0:
                    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
                else:
                    model.add(ZeroPadding2D((1,1)))

            print("Convolution2D(",number_of_filters,", (",filter_x,", ",filter_y,"), name='",mat_model_layer,"')")
            model.add(Convolution2D(number_of_filters, (filter_x, filter_y), name= mat_model_layer))

        else:
            if mat_model_layer.find("relu") == 0:
                print("Activation('relu', name=",mat_model_layer)
                model.add(Activation('relu', name=mat_model_layer))
            elif mat_model_layer.find("dropout") == 0:
                print("Dropout(0.5, name=",mat_model_layer,")")
                model.add(Dropout(0.5, name=mat_model_layer))
            elif mat_model_layer.find("pool") == 0:
                print("MaxPooling2D((2,2), strides=(2,2), name=",mat_model_layer,")")
                model.add(MaxPooling2D((2,2), strides=(2,2), name=mat_model_layer))
            elif mat_model_layer.find("softmax") == 0:
                print("Activation('softmax', name=",mat_model_layer,")")
                model.add(Activation('softmax', name=mat_model_layer))
            else:
                print("-->",mat_model_layer)
    return model

Save model to .h5 file for use later.

In [4]:
model = vgg_tf()
model.save('data/vgg_face.h5')

ZeroPadding2D((1,1))
Convolution2D( 64 , ( 3 ,  3 ), name=' conv1_1 ')
Activation('relu', name= relu1_1
ZeroPadding2D((1,1))
Convolution2D( 64 , ( 3 ,  3 ), name=' conv1_2 ')
Activation('relu', name= relu1_2
MaxPooling2D((2,2), strides=(2,2), name= pool1 )
ZeroPadding2D((1,1))
Convolution2D( 128 , ( 3 ,  3 ), name=' conv2_1 ')
Activation('relu', name= relu2_1
ZeroPadding2D((1,1))
Convolution2D( 128 , ( 3 ,  3 ), name=' conv2_2 ')
Activation('relu', name= relu2_2
MaxPooling2D((2,2), strides=(2,2), name= pool2 )
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_1 ')
Activation('relu', name= relu3_1
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_2 ')
Activation('relu', name= relu3_2
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_3 ')
Activation('relu', name= relu3_3
MaxPooling2D((2,2), strides=(2,2), name= pool3 )
ZeroPadding2D((1,1))
Convolution2D( 512 , ( 3 ,  3 ), name=' conv4_1 ')
Activation('relu', name= relu4_1
ZeroPadding2D((

## Load Dataset

In [5]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from PIL import Image
import numpy as np
from tqdm import tqdm
import os

In [6]:
def pad(img):
    ht, wd, cc= img.shape
    ww = 224
    hh = 224
    color = (0,0,0)
    result = np.full((hh,ww,cc), color, dtype=np.uint8)
    xx = (ww - wd) // 2
    yy = (hh - ht) // 2
    result[yy:yy+ht, xx:xx+wd] = img
    return result

def load_img_file(path, model):
    img = Image.open(path)
#     img = load_img(path)
    x = img_to_array(img) 
    x = pad(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x = model(x)
    return x

def load_data(path, model):
    x = []
    y = []
    labels = {}
    folders = os.listdir(path)
    for i in tqdm(range(len(folders))):
        labels[i] = folders[i]
        label_path = path + '/' + folders[i]
        images = os.listdir(label_path)
        for image in images:
            img = load_img_file(label_path + '/' + image, model)
            x.append(np.squeeze(K.eval(img)).tolist())
            y.append(i)
    return x, y, labels

In [7]:
X_train = y_train = labels = None
try:
    X_train = np.load('data/combined/X_train.npy', allow_pickle=True)
    y_train = np.load('data/combined/y_train.npy', allow_pickle=True)
    labels = np.load('data/combined/labels.npy', allow_pickle=True)[()]
except:
    X_train, y_train, labels = load_data('data/combined/aligned', model)
    np.save('data/combined/X_train', X_train)
    np.save('data/combined/y_train', y_train)
    np.save('data/combined/labels', labels)

In [8]:
X_test = y_test = None
try:
    X_test = np.load('data/combined/X_test.npy', allow_pickle=True)
    y_test = np.load('data/combined/y_test.npy', allow_pickle=True)
    labels = np.load('data/combined/labels.npy', allow_pickle=True)[()]
except:
    X_test, y_test, labels = load_data('data/combined/valid', model)
    np.save('data/combined/X_test', X_test)
    np.save('data/combined/y_test', y_test)

In [9]:
print(len(labels.keys()))

140


## Construct Classifier 

In [20]:
def classier_model(input_dim, num_classes):
    classifier = Sequential()
    classifier.add(Dense(500, input_dim=input_dim))
    classifier.add(BatchNormalization())
    classifier.add(Activation('relu'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(250))
    classifier.add(BatchNormalization())
    classifier.add(Activation('relu'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(num_classes))
    classifier.add(Activation('softmax'))
    classifier.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='nadam', metrics=['accuracy'])
    return classifier

## Train Classifier

In [21]:
from matplotlib import pyplot as plt

def plotter(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

### Gender Classification
Train classifier for gender classification (of all ages together).

In [ ]:
y_train_gender = y_train
y_test_gender = y_test
y_train_gender[y_train_gender % 2 == 0] = 0
y_train_gender[y_train_gender % 2 != 0] = 1
y_test_gender[y_test_gender % 2 == 0] = 0
y_test_gender[y_test_gender % 2 != 0] = 1

classifier = classier_model(2622, 2)
history = classifier.fit(X_train, y_train_gender, batch_size=64, epochs=100, validation_data=(X_test, y_test_gender))
plotter(history)

Epoch 1/100
387/460 [========================>.....] - ETA: 0s - loss: 0.5933 - accuracy: 0.6720

Train classifier on all the classes.

In [12]:
classifier = classier_model(2622, 140)
history = classifier.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
190/460 [===========>..................] - ETA: 3s - loss: 1.1001 - accuracy: 0.5697

KeyboardInterrupt: 

Train classifier on each class.

In [ ]:
def train():
    acc = {}
    for i in range(0, 70, 2):
        classifier = classier_model(2622, 2)
        female_train = np.where(y_train == i)[0]
        female_test = np.where(y_test == i)[0]
        male_train = np.where(y_train == i+1)[0]
        male_test = np.where(y_test == i+1)[0]

        X_train_label = np.concatenate((X_train[female_train], X_train[male_train]), axis=0)
        y_train_label = np.concatenate((y_train[female_train], y_train[male_train]), axis=0) 
        X_test_label = np.concatenate((X_test[female_test], X_test[male_test]), axis=0)
        y_test_label = np.concatenate((y_test[female_test], y_test[male_test]), axis=0)
        print(len(X_train_label))
        print(len(y_train_label))
        print(len(X_test_label))
        print(len(y_test_label))

        history = classifier.fit(X_train_label, y_train_label, epochs=100, validation_data=(X_test_label,y_test_label), verbose=0)
        train_acc = history.history['accuracy'][-1]
        val_acc = history.history['val_accuracy'][-1]
        acc[i] = {'train': train_acc, 'val': val_acc}
        print(acc[i])
    return acc

acc = train()

In [ ]:
print(acc)

## Results